<h1><center>Northeastern University</center></h1>
<h1><center>EECE 7150 Autonomous Field Robotics</center></h1>
<h1><center>HW3 Submission</center></h1>
<h3><center>Yash Mewada</center></h3>
<h3><center>Date: 25th Sept, 2023</center></h3>

<h3>Part 1</h3>


Modify your registration module from the previous home works to allow you to robustly match features across two images. Please continue to work with Python Notebooks as much as possible.

In [411]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
from operator import sub
import gtsam
import gtsam.utils.plot as gtsam_plot
import os
import math


class hw3():
    def __init__(self,path_to_dataset):
        """
        Constructor for hw3 class
        Args:
            path_to_dataset (str): path to dataset
        """
        self.path_to_dataset = path_to_dataset
    
    def pixelCoorNormalise(self,coor,img)->tuple:
        """
        Normalise pixel coordinate to range [-1,1]
        Args:
            coor (tuple): (x,y) coordinate
            img (np.array): image

        Returns:
            tuple: (x,y) coordinate normalised
        """
        x,y = coor
        h,w = img.shape[:2]
        return (2*x/w - 1,2*y/h - 1)

    def pixelCoorDenormalise(self,coor,img)->tuple:
        """
        Denormalise pixel coordinate to range [0,255]
        Args:
            coor (tuple): (x,y) coordinate
            img (np.array): image

        Returns:
            tuple: (x,y) coordinate denormalised
        """
        x,y = coor
        h,w = img.shape[:2]
        return (float((x+1)*w/2),float((y+1)*h/2))
    
    def performCLAHE(self,img):
        """
        Perform CLAHE on image
        Args:
            img (np.array): image
        """
        assert img is not None, "file could not be read, check with os.path.exists()"
        # create a CLAHE object (Arguments are optional).
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl1 = clahe.apply(img)
        return cl1
        
        
    def siftDetctor(self,img1,img2):
        """
        sift feature detector

        Args:
            img1 (np.array): image 1 
            img2 (np.array): image 2

        Returns:
            img3 (np.array): image with matches
            kp1 (list): Normalised list of keypoints in image 1
            kp2 (list): Normalised list of keypoints in image 2
        """
        
        img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        
        # img1 = self.performCLAHE(img1)
        # img2 = self.performCLAHE(img2)
        
        sift = cv2.SIFT_create()
        kp1,des1 = sift.detectAndCompute(img1,None)
        kp2,des2 = sift.detectAndCompute(img2,None)
    
        # Brute Force Matching
        matches = sorted(cv2.BFMatcher(cv2.NORM_L1,crossCheck=True).match(des1,des2),key=lambda x:x.distance)
        goodkp2 = [kp2[mat.trainIdx] for mat in matches]
        goodkp1 = [kp1[mat.queryIdx] for mat in matches]
        #print(goodkp1[0].pt)

        img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:50],None,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        return img3,goodkp1,goodkp2
    
    def PoissonImageBlending(self,source, destination):
        # create an all "White" mask: 255, if black mask is 0
        mask = 255 * np.ones(destination.shape, destination.dtype) 
        # navigate the source img location
        width, height, channels = source.shape
        center = (height//2, width//2)

        # using built-in funtion `cv2.seamlessClone` to acommpulish Poisson Image
        blended = cv2.seamlessClone(destination, source, mask, center, 3) # cv::MIXED_CLONE = 2
        output = blended
        return output
    
    def featherBlending(self,A,B):
        """
        Feather gradeint blending
        """
        assert A is not None, "file could not be read, check with os.path.exists()"
        assert B is not None, "file could not be read, check with os.path.exists()"
        # generate Gaussian pyramid for A
        G = A.copy()
        gpA = [G]
        for i in range(6):
            G = cv2.pyrDown(G)
            gpA.append(G)
        # generate Gaussian pyramid for B
        G = B.copy()
        gpB = [G]
        for i in range(6):
            G = cv2.pyrDown(G)
            gpB.append(G)
        # generate Laplacian Pyramid for A
        lpA = [gpA[5]]
        for i in range(5,0,-1):
            GE = cv2.pyrUp(gpA[i])
            L = cv2.subtract(gpA[i-1],GE)
            lpA.append(L)
        # generate Laplacian Pyramid for B
        lpB = [gpB[5]]
        for i in range(5,0,-1):
            GE = cv2.pyrUp(gpB[i])
            L = cv2.subtract(gpB[i-1],GE)
            lpB.append(L)
        # Now add left and right halves of images in each level
        LS = []
        for la,lb in zip(lpA,lpB):
            rows,cols,dpt = la.shape
            ls = np.hstack((la[:,0:cols//2], lb[:,cols//2:]))
            LS.append(ls)
        # now reconstruct
        ls_ = LS[0]
        for i in range(1,6):
            ls_ = cv2.pyrUp(ls_)
            ls_ = cv2.add(ls_, LS[i])
        # image with direct connecting each half
        real = np.hstack((A[:,:cols//2],B[:,cols//2:]))
        return ls_,real

    def computeHomography(self,kp1,kp2,img1,img2):
        """
        Compute homography matrix

        Args:
            kp1 (list): Normalised list of keypoints in image 1
            kp2 (list): Normalised list of keypoints in image 2

        Returns:
            H (np.array): Homography matrix
        """
        kp1 = np.array([kp.pt for kp in kp1])
        kp2 = np.array([kp.pt for kp in kp2])
        H,mask = cv2.findHomography(kp2,kp1,cv2.RANSAC)
        # H,mask  = cv2.estimateAffinePartial2D(kp1,kp2)
        # # H = H[:2,:]
        # homogenous = np.array([[0,0,1]])
        # H = np.concatenate((H,homogenous),axis=0)
        # mathchesMask = mask.ravel().tolist()
        return H

    def computeAffine(self,kp1,kp2):
        """
        Compute affine matrix
        Args:
            kp1 (_type_): _description_
            kp2 (_type_): _description_
            img1 (_type_): _description_
            img2 (_type_): _description_
        """
        kp1 = np.array([kp.pt for kp in kp1])
        kp2 = np.array([kp.pt for kp in kp2])
        H,_  = cv2.estimateAffinePartial2D(kp2,kp1)
        H = H[:2,:]
        homogenous = np.array([[0,0,1]])
        H = np.concatenate((H,homogenous),axis=0)
        
        # scale = np.sqrt(H[0,0]**2 + H[1,0]**2)
        # H[:2,:2] /= scale
        affine_mat = H
        return affine_mat
    
    def overlay_two_images(self,img1,img2,ignore_color = [0,0,0]):
        ignore_color = np.asarray(ignore_color)
        mask = (img2==ignore_color).all(-1,keepdims=True)
        out = np.where(mask,img1,(img1 * 0.5 + img2 * 0.5).astype(img1.dtype))
        return out
    
    def mean_blend(self,img1, img2):
        assert(img1.shape == img2.shape)
        locs1 = np.where(cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY) != 0)
        blended1 = np.copy(img2)
        blended1[locs1[0], locs1[1]] = img1[locs1[0], locs1[1]]
        locs2 = np.where(cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY) != 0)
        blended2 = np.copy(img1)
        blended2[locs2[0], locs2[1]] = img2[locs2[0], locs2[1]]
        blended = cv2.addWeighted(blended1, 0.8, blended2, 0.2, 0)
        # blended = self.PoissonImageBlending(blended1,blended2)
        return blended
    
    def warpImage(self,img1,img2,H,height,width,crop_black=True):
        """
        Warp image 1 to image 2
        """
        
        h,w = img1.shape[:2]
        pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        dst = cv2.warpPerspective(img1,H,(width,height))
        #print(img2.shape)
        # blended = self.PoissonImageBlending(dst,img2)sss
        
        # blended = cv2.addWeighted(img2,0.5,dst,0.5,0)
        blended = self.mean_blend(img2,dst)
        final = np.zeros((img2.shape[0] + img2.shape[0],img2.shape[1] + img2.shape[1],3),np.uint8)
        # # find center of the final canvas
        
        if crop_black:
            final[0:img2.shape[0],0:img2.shape[1]] = blended
            x,y,w,h = cv2.boundingRect(cv2.findNonZero(cv2.cvtColor(final, cv2.COLOR_BGR2GRAY)))
            final = final[y:y+h,x:x+w]
        
        else:
            (cx,cy) = (final.shape[1]//2,final.shape[0]//2)
            # find center of the blended image
            (bx,by) = (blended.shape[1]//2,blended.shape[0]//2)
            # merge the blended image to the final canvas such that the center of the blended image is at the center of the final canvas
            final[cy-by:cy-by+blended.shape[0],cx-bx:cx-bx+blended.shape[1]] = blended
        # final = np.asarray(np.nonzero(final),dtype=np.uint8)
        # #print(h,w)

        return blended
    

        
        


In [412]:
class colors:
    GREEN = '\033[32m'
    BLUE = '\033[34m'
    YELLOW = '\033[33m'
    RESET = '\033[0m'

In [413]:
def findHallimgs(hw3:hw3,dataset:str,image29=False):
    
    image_lsit = []
    # dataset = os.path.join(hw3.path_to_dataset,dataset)
    # files = os.listdir(dataset)
    # files.sort()
    
    if image29:
        files = []
        for root, dirs, f in os.walk(dataset):
            for file in f:
                # Get the full path of the file
                file_path = os.path.abspath(os.path.join(root, file))
                print(file_path)
                files.append(file_path)
    else:
        dataset = os.path.join(hw3.path_to_dataset,dataset)
        files = os.listdir(dataset)
    files.sort()

    for file,i in zip(files,range(len(files))):
        if i < 2:
            img = cv2.imread(os.path.join(dataset,file),cv2.IMREAD_GRAYSCALE)
            if image29:
                img = cv2.resize(img,(0,0),fx=1,fy=1)
            clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(5,5))
            cl1 = clahe.apply(img)
            cl1 = cv2.cvtColor(cl1,cv2.COLOR_GRAY2BGR)
            image_lsit.append(cl1)
        else:
            if image29:
                img = cv2.resize(img,(0,0),fx=0.5,fy=0.5)
            img = cv2.imread(os.path.join(dataset,file))
            image_lsit.append(img)
        
    H_d = {}
    n_features = {}
    aff1 = {}
    for i in range(len(image_lsit)-1):
        key = str(i)+str(i+1)
        # print(colors.BLUE+"Computing Homography for image pair: "+key+colors.RESET)
        _,kp1,kp2 = hw3.siftDetctor(image_lsit[i],image_lsit[i+1])
        n_features[i] = len(kp1)
        H = hw3.computeHomography(kp1,kp2,image_lsit[i],image_lsit[i+1])
        aff1[key] = hw3.computeAffine(kp1,kp2)
        H_d[key] = H
    # print(n_features.keys())
    print("Consecutiive frames H:",H_d.keys()) 
    used_keys = []
    H_new = {}

                    
    aff = {}
    H_init = H_d['01']
    i = 0
    for key1 in H_d:
        for key2 in H_d:
            new_key = key1[0] + key2[1]  # Generate the new key
            swapped_key = key2[0] + key1[1]  # Generate the swapped key
            # Check conditions to ensure that swapped keys and duplicates are avoided
            if (key1 != key2) and (new_key not in used_keys) and (swapped_key not in used_keys):
                # print("Non consecutive frames")
                cumulative_H = H_init
                current_key = key1
                while current_key[1] != key2[1]:
                    next_key = f"{current_key[1]}{str(int(current_key[1]) + 1)}"
                    cumulative_H = np.dot(cumulative_H,H_d[next_key])  # Adjust this based on your key format
                    cumulative_H[2,2] = 1
                    current_key = next_key
                #     print("inside",i)
                # print("out")
                _,kp1,kp2 = hw3.siftDetctor(image_lsit[int(key1[0])],image_lsit[int(key2[1])])
                aff[new_key] = hw3.computeAffine(kp1,kp2)
                # H_new[new_key] = hw3.computeHomography(kp1,kp2,image_lsit[int(key1[0])],image_lsit[int(key2[1])])
                # print(f"{colors.YELLOW}Affine matrix for image pair: {key1[0]}"  "{key2[1]} computed{colors.RESET}")
                H_new[new_key] = cumulative_H
                used_keys.append(new_key)
                i += 1
                
    aff.update(aff1)
    H_d.update(H_new)
    
    h00 = np.array([[1,0,0],[0,1,0],[0,0,1]],dtype=np.float32)
    my_h = {'00':h00}
    my_a = {'00':h00}
    for keys,values in H_d.items():
        if keys[0] == '0':
            my_h[keys] = values
    
    for keys,values in aff.items():
        if keys[0] == '0':
            my_a[keys] = values
    # print(f"{colors.GREEN}Homography matrix for all image pairs computed{colors.RESET}",H.keys())
    print(f"{colors.GREEN}Homography matrix for all image pairs computed{colors.RESET}",H_d.keys())
    print(f"{colors.GREEN}Affine matrix for all image pairs computed{colors.RESET}",aff.keys())
    print(f"{colors.GREEN}Only H{colors.RESET}",my_h.keys())

    return H_d,image_lsit,my_h,n_features,aff
    

In [414]:
def addBorder(img, rect):
    top, bottom, left, right = int(0), int(0), int(0), int(0)
    x, y, w, h = rect
    tl = (x, y)    
    br = (x + w, y + h)
    if tl[1] < 0:
        top = -tl[1]
    if br[1] > img.shape[0]:
        bottom = br[1] - img.shape[0]
    if tl[0] < 0:
        left = -tl[0]
    if br[0] > img.shape[1]:
        right = br[0] - img.shape[1]
    img = cv2.copyMakeBorder(img, top, bottom, left, right,
                            cv2.BORDER_CONSTANT, value=[0, 0, 0])
    orig = (left, top)
    return img, orig

def patchPano(hw3:hw3,img1, img2, orig1=(0,0), orig2=(0,0)):
    # bottom right points
    br1 = (img1.shape[1] - 1, img1.shape[0] - 1)
    br2 = (img2.shape[1] - 1, img2.shape[0] - 1)
    # distance from orig to br
    diag2 = tuple(map(sub, br2, orig2))
    # possible pano corner coordinates based on img1
    extremum = np.array([(0, 0), br1,
                tuple(map(sum, zip(orig1, diag2))),
                tuple(map(sub, orig1, orig2))])
    bb = cv2.boundingRect(extremum)
    # patch img1 to img2
    pano, shift = addBorder(img1, bb)
    orig = tuple(map(sum, zip(orig1, shift)))
    idx = np.s_[orig[1] : orig[1] + img2.shape[0] - orig2[1],
                orig[0] : orig[0] + img2.shape[1] - orig2[0]]
    subImg = img2[orig2[1] : img2.shape[0], orig2[0] : img2.shape[1]]
    pano[idx] = hw3.mean_blend(pano[idx], subImg)
    # pano[idx] = hw3.featherBlending(pano[idx], subImg)
    return np.array(pano), orig

In [415]:
def dimFind(img,H):
    transPts=[]
    img1=img
    H1=H
    h1,w1 = img1.shape[:2] #grab height and width from input image
    pts1 = ((0,0),(w1,0),(w1,h1),(0,h1)) #create an array of just the corners in order bottom left, top left, top right, bottom right

    for pt in pts1:
        transPts.append(np.matmul(H1,np.array([pt[0],pt[1],1]))) #matmul the corners by the homography

    #this little x_min/max section was inspired by the great Xavier Hubbard 
    x_min = min(pt[0]/pt[2] for pt in transPts)
    x_max = max(pt[0]/pt[2] for pt in transPts)
    #doing the same for y, I think it makes the pano look worse though...
    y_min = min(pt[1]/pt[2] for pt in transPts)
    y_max = max(pt[1]/pt[2] for pt in transPts)
    

    if x_min<0:
        x_shift = abs(x_min)
    else:
        x_shift = 0

    if y_min<0:
        y_shift = abs(y_min)
    else:
        y_shift = 0

    transH = np.array([[1,0,np.ceil(x_shift)],[0,1,0],[0,0,1]])

    dim = (int(x_max+x_shift),int(y_max+y_shift))
    return dim, transH

def cavas_dim(image_list,H):
    min_x = float('inf')
    max_x = -float('inf')
    min_y = float('inf')
    max_y = -float('inf')
    
    for i,value in zip(range(len(image_list)),H.values()):
        h,w,_ = image_list[i].shape
        corners = np.float32([[0,0],[w,0],[w,h],[0,h]])
        transformed_corners = cv2.perspectiveTransform(corners.reshape(-1,1,2),value).reshape(-1,2)
        corners_tf = np.vstack((transformed_corners,corners.reshape(-1,2)))
        
        min_x_i = int(np.min(corners_tf[:, 0]))
        max_x_i = int(np.max(corners_tf[:, 0]))
        min_y_i = int(np.min(corners_tf[:, 1]))
        max_y_i = int(np.max(corners_tf[:, 1]))

        # Update the overall minimum and maximum coordinates
        min_x = min(min_x, min_x_i)
        max_x = max(max_x, max_x_i)
        min_y = min(min_y, min_y_i)
        max_y = max(max_y, max_y_i)
    
    return min_x,max_x,min_y,max_y
    

def getShift(image,H):
    corners = np.float32([[0,0],[0,image.shape[0]-1],[image.shape[1]-1,image.shape[0]-1],[image.shape[1]-1,0]])
    H_corners = []
    # transform these point to the respective homography
    for p in corners:
        # #print(p)
        p = np.array([p[0],p[1],1])
        p = np.dot(H,p)
        p = p/p[2]
        H_corners.append(p)
    
    shift = (float("inf"),float("inf"))
    shift_x = 0
    shift_y = 0
    for p in H_corners:
        # get the least negative x and y
        if p[0] < shift[0]:
            shift_x = int(p[0])
        if p[1] < shift[1]:
            shift_y = int(p[1])
        else:
            shift_x = 0
            shift_y = 0
        shift = (shift_x,shift_y)

    return shift    
    
def stitchformHdynamic(hw3:hw3,H,image_list):
    # stich the images based on the dictionary of homography matrices
    
    min_x,max_x,min_y,max_y = cavas_dim(image_list,H)
    # odom = []
    print("min_x,max_x,min_y,max_y",min_x,max_x,min_y,max_y)
    canvas_width = max_x - min_x
    canvas_height = max_y - min_y
    print("canvas_width,canvas_height",canvas_width,canvas_height)
    
    if canvas_height>10e4 or canvas_width>10e4:
        return np.zeros((100, 100, 3), np.uint8)
    
    canvas = np.zeros((canvas_height,canvas_width,3),np.uint8)
    tx = -min_x
    ty = -min_y
    t_mat = np.array([[1,0,tx],[0,1,ty],[0,0,1]],dtype=np.float32)
    
    print("H length",len(H))
    for i,value in zip(range(len(image_list)),H.values()):
        # print("i",i)
        # if i == 0:
        #     canvas = cv2.warpPerspective(image_list[0],t_mat,(canvas_width,canvas_height))
        # else:
        H_new = t_mat@value
        warped_image = cv2.warpPerspective(image_list[i],H_new,(canvas_width,canvas_height))
        
        canvas = hw3.mean_blend(canvas,warped_image)
        # H_new = t_mat@value
        # warped_image = cv2.warpPerspective(image_list[i],H_new,(canvas_width,canvas_height))
        
        # canvas = hw3.mean_blend(canvas,warped_image)
    
    x,y,w,h = cv2.boundingRect(cv2.findNonZero(cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)))
    return canvas[y:y+h,x:x+w],t_mat

**Feature Detector** - In part 1, I found that Clahe only worked for me for inital 2/3 images, hence I only applied CLAHE to those images.This code uses Brute Force Matcher (BF Matcher) to find the matches and get the good matches out of those. I am using SIFT detector here. I also tested it with ORB, but the output by ORB gets worse after 4 images, (which should not be the case as in certain papers they claim that ORB outperforms SIFT because of its bitwise descriptor).

**RANSAC** - The Homography matrix estimation is done by RANSAC. I also tried estimating the affine transformation matrix,but the results were a bit off.

**Image Stitching and Panorama generation** - For panorama generation and estimating the location of image before sticching, I take four corners of the image and then multiply it with its respective H matrix. Then in those four warped points if any of the *x,y* is out of canvas i.e negative, I shit the previous panorama/image by the absolute value of that shift i.e (np.roll), hence giving me shift in X and Y axes. After the shifts are available I warp and stitch the images.

**Mean/Alpha Bleding** - I found while naively applying alpha blending is it also blends the black pixels of the warped image hence making the blended image darker and darker as the number of images increases, hence I applied what I call a "*Mean Blending*", here I copy the non zeros pixels of both the images and then perform Alpha blending hence, only the non zero pixels gets blended.

**Note** - The implementation of all above methods is in *main* function below.

<h3>Part 2</h3>
Go around the entire loop of 6 images and set up the problem by outputting vertices and edges in terms of a factor graph

**Pose 2 genetation** - We all know that the H matrix is a combination of camera intrinsics, Rotation and Translation matrix as.
$\mathbf{H} = \mathbf{K} \cdot \mathbf{R} \cdot \mathbf{T}$, Where:
\begin{align*}
    \mathbf{K} & : \text{Intrinsic camera matrix} \\
    \mathbf{R} & : \text{Extrinsic rotation matrix} \\
    \mathbf{T} & : \text{Extrinsic translation vector} \\
\end{align*}
Hence we can obtain the tranlation from the right top most element of the H matrix as:
$\mathbf{H} =
\begin{bmatrix}
    h_{11} & h_{12} & x \\
    h_{21} & h_{22} & y \\
    h_{31} & h_{32} & 1 \\
\end{bmatrix}$
Then I did SVD of the top left matrix of this H matrix to obtaint the Rotation matrix and then $\tan^{-1}$ will give me the pose of the image. Although there was no difference in this case where we directly take the top left corner of the H matrix to compute the value of $\theta$ or decompose it and then take proper rotation matrix because ther vertices are just points, but decomposing the top left of H matrix to obtain R matrix made sense theoretically.

This pose is then further given to the factor graph to generate the graph and then eventually optimise it using *Levenberg-Marquardt* algorithm.
**Note** - Plot graph function is taken from ZZ's code.



In [416]:
def create_pose2(hw3:hw3,dataset,image29=False,t_mat=None):
    H_dict,image_list,_,n,_ = findHallimgs(hw3,dataset,image29)
    MAX_MATCHES = max(n.values())
    # #print("H",H)
    w,h = image_list[0].shape[1],image_list[0].shape[0]
        
    #prior mean and noise model and taking center of the image as the prior pose
    priorMean = gtsam.Pose2(w/2,h/2,0)
    PRIOR_NOISE_MODEL = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
    ODOM_NOISE_MODEL = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
    
    graph = gtsam.NonlinearFactorGraph()
    graph.add(gtsam.PriorFactorPose2(0, priorMean, PRIOR_NOISE_MODEL))
    
    initial = gtsam.Values()
    initial.insert(0, gtsam.Pose2())
    i = 1
    
    for key, value in H_dict.items():
        if key.startswith('0'):
            H = value
            x,y = H[0,-1],H[1,-1]
            U, _, Vt = np.linalg.svd(H[:2, :2])
            R = np.dot(U, Vt)
            theta = np.arctan2(H[1,0],H[0,0])
            pose = gtsam.Pose2(x,y,theta)
            initial.insert(i, pose)
            i += 1
    
    def getNoise(x,y,th,n):
        # A part of this function is implemented form Curtis's Code
        ERF_DIV = 50
        LIN_DIV = 50
        covar_multiplier = math.exp(n - MAX_MATCHES)
        exx = abs(x / LIN_DIV) * covar_multiplier
        eyy = abs(y / LIN_DIV) * covar_multiplier
        ett = abs(th / (LIN_DIV * 10.0)) * covar_multiplier
        # covar_multiplier = 1/math.erf((n-3)/ERF_DIV)

        # exx = abs(x/LIN_DIV) * covar_multiplier
        # eyy = abs(y/LIN_DIV) * covar_multiplier
        # ett = abs(th/(LIN_DIV * 10.0)) * covar_multiplier
        
        diag_noise = np.array([exx, eyy, ett])
        return gtsam.noiseModel.Diagonal.Sigmas(gtsam.Point3(diag_noise))
    
    i = 0
    for key,value in H_dict.items():
        H = value
        x,y = H[0,-1],H[1,-1]
        U, _, Vt = np.linalg.svd(H[:2, :2])
        R = np.dot(U, Vt)
        theta = np.arctan2(R[1,0],R[0,0])
        src = int(key[0])
        dst = int(key[1])
        if i < 5:
            ODOM_NOISE_MODEL = getNoise(x,y,theta,n[i])
        else:
            ODOM_NOISE_MODEL = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))    
        graph.add(gtsam.BetweenFactorPose2(src,dst,gtsam.Pose2(x,y,theta),ODOM_NOISE_MODEL))
        i += 1

    #print("\nFactor Graph:\n{}".format(graph))
    return graph,initial

def plot_graph(values,graph,marginals,res):
    #print("Values:\n{}".format(values))
    #print("Marginals:\n{}".format(marginals))
    for vertex_index in range(values.size()):
        gtsam_plot.plot_pose2(0, values.atPose2(
            vertex_index), 0.5, marginals.marginalCovariance(vertex_index))

    for edge_index in range(graph.size())[1:]:
        key1, key2 = graph.at(edge_index).keys()

        start_pose = values.atPose2(key1)
        end_pose = values.atPose2(key2)
        plt.plot([start_pose.x(), end_pose.x()],
                 [start_pose.y(), end_pose.y()],
                 color='blue')
    plt.title(res)
    plt.grid()
    plt.savefig(res+".png")
    plt.show()

def reconstructH(result,image_list,prior,t_mat):
    poses = gtsam.utilities.allPose2s(result)
    H_dict = {}
    w,h = image_list[0].shape[1],image_list[0].shape[0]
    # print(poses.size())affine_H1
    for i in range(poses.size()):
        key = str(i)+str(i+1)
        pose = poses.atPose2(i)
        affine_H = t_mat@pose.matrix()
        # if i == 0:
        #     affine_H = t_mat@affine_H
        # if prior == False:
        #     trans = np.eye(3)
        #     trans[:2,2] -= np.array([w/2, h/2]).T
        #     affine_tf = affine_H@trans
        #     # if i > 0:
        #     H_dict[key] = affine_tf
        # else:
        H_dict[key] = affine_H
        # H_dict[key] = affine_H
    print("reconstructed keys",H_dict.keys())
    return H_dict


The way I have calculated the covariance is based on the number of matches between the images, the more the matches the less the covariance. I have used the following formula to calculate the covariance:
\begin{align*}
    \text{covar\_multiplier} = \exp(n - \text{MAX\_MATCHES})\\
    exx = \left| \frac{x}{30} \right| \cdot \text{covar\_multiplier}\\
    eyy = \left| \frac{y}{30} \right| \cdot \text{covar\_multiplier}\\
    ett = \left| \frac{\theta}{30 \cdot 10.0} \right| \cdot \text{covar\_multiplier}\\
    \text{diag\_noise} = [ \text{exx}, \text{eyy}, \text{ett} ]
\end{align*}
The above expression is called exponential scaling of covariance based on the number of matches. Exponential scaling, which can provide more drastic adjustments based on the number of matches.


LIN_DIV is a constant or parameter that is used to divide the displacement values dx, dy, and dth in order to normalize them before applying the scaling factor covar_multiplier. It's essentially a divisor used to adjust the sensitivity of the covariance scaling based on the size of the displacement.

The choice of LIN_DIV should be based on the scale and units of your motion or displacement measurements. It can be used to control how much the covariance scales with respect to the magnitude of motion. A smaller LIN_DIV will result in larger covariances for the same magnitude of motion, while a larger LIN_DIV will result in smaller covariances.


In [417]:
def images_6():
    print(f"{colors.BLUE}OUTPUT for 6 images{colors.RESET}")
    dataset = "/home/mewada/Documents/AFR/eece7150/HW3/Dataset/6Images/"
    hw31 = hw3(dataset)
    H_d,image_list,H,_,_ = findHallimgs(hw31,dataset,image29=False)
    
    pano,t_mat = stitchformHdynamic(hw31,H,image_list)
    cv2.imshow("pano",pano)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(f"{colors.BLUE}Before Optimisation{colors.RESET}")
    graph,initial = create_pose2(hw31,dataset,image29=False,t_mat=t_mat)
    print(f"{colors.BLUE}Initials{colors.RESET}",initial)
    marginals = gtsam.Marginals(graph,initial)
    res = "Before Optimisation"
    plot_graph(initial,graph,marginals,res)
    initial_h = reconstructH(initial,image_list,prior=True,t_mat=t_mat)
    pano_before_opt,_ = stitchformHdynamic(hw31,initial_h,image_list)
    
    cv2.imshow("pano_before_opt",pano_before_opt)
    # plt.imshow(pano_before_opt)
    # plt.title(res)
    # plt.axis('equal')
    # plt.savefig("pano_before_opt.png")
    # plt.show()
    # plt.close()
    
    print(f"{colors.GREEN}After Optimisation{colors.RESET}")
    params = gtsam.LevenbergMarquardtParams()
    optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)
    result = optimizer.optimize()
    res = "After Optimisation"
    print(f"{colors.BLUE}Result{colors.RESET}",result)
    marginals = gtsam.Marginals(graph, result)
    plot_graph(result, graph, marginals,res)
    result_h = reconstructH(result,image_list,prior=True,t_mat=t_mat)
    
    pano_after_opt,_ = stitchformHdynamic(hw31,result_h,image_list)
    cv2.imshow("pano_after_opt",pano_after_opt)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # plt.imshow(pano_after_opt)
    # plt.title(res)
    # plt.axis('equal')
    # plt.savefig("pano_after_opt.png")
    # plt.show()
    # plt.close()

In [418]:
def images_29():
    print(f"{colors.BLUE}OUTPUT for 6 images{colors.RESET}")
    dataset = "/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/"
    hw31 = hw3(dataset)
    H_d,image_list,H,_,_ = findHallimgs(hw31,dataset,image29=True)
    
    pano,t_mat = stitchformHdynamic(hw31,H,image_list)
    cv2.imshow("pano",pano)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(f"{colors.BLUE}Before Optimisation{colors.RESET}")
    graph,initial = create_pose2(hw31,dataset,image29=True,t_mat=t_mat)
    print(f"{colors.BLUE}Initials{colors.RESET}",initial)
    marginals = gtsam.Marginals(graph,initial)
    res = "Before Optimisation"
    plot_graph(initial,graph,marginals,res)
    initial_h = reconstructH(initial,image_list,prior=True,t_mat=t_mat)
    pano_before_opt,_ = stitchformHdynamic(hw31,initial_h,image_list)
    
    cv2.imshow("pano_before_opt",pano_before_opt)
    # plt.imshow(pano_before_opt)
    # plt.title(res)
    # plt.axis('equal')
    # plt.savefig("pano_before_opt.png")
    # plt.show()
    # plt.close()
    
    print(f"{colors.GREEN}After Optimisation{colors.RESET}")
    params = gtsam.LevenbergMarquardtParams()
    optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)
    result = optimizer.optimize()
    res = "After Optimisation"
    print(f"{colors.BLUE}Result{colors.RESET}",result)
    marginals = gtsam.Marginals(graph, result)
    plot_graph(result, graph, marginals,res)
    result_h = reconstructH(result,image_list,prior=True,t_mat=t_mat)
    
    pano_after_opt,_ = stitchformHdynamic(hw31,result_h,image_list)
    cv2.imshow("pano_after_opt",pano_after_opt)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # plt.imshow(pano_after_opt)
    # plt.title(res)
    # plt.axis('equal')
    # plt.savefig("pano_after_opt.png")
    # plt.show()
    # plt.close()

In [419]:
    
if __name__ == "__main__":
    images_6()
    # images_29()
    
    


    
        

OUTPUT for 6 images
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/4.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/6.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/2.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/3.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/7.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/1.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/5.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/fourth column/8.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/second column/ESC.970622_025500.0621.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/second column/ESC.970622_025447.0620.tif
/home/mewada/Documents/AFR/eece7150/HW3/Dataset/29images/29images/second colum

It is seen from the above output that the factor graph optimization worked but not significantly in 6 images case as the initial estimates were good but there is a slight decrease in errors as the graph optimises.
I tried it with 29 images, but the kernel was crashing as I was trying to process all the images at once. If time persists I will try and upload the output of all 29 images, but for now I tried just the third column of the 29 images.

Also the factor graph optimization is done, but the plotting of images after was a bit difficult as I wanted to experiment with a dynamically chaning canvas size rather than any constant valued.
